In [1]:
# Packages
import dask.dataframe as dd
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

In [ ]:
import psutil

# Obtenir les informations sur la mémoire
memory_info = psutil.virtual_memory()

# Afficher la mémoire totale, utilisée et disponible en GB
total_memory = memory_info.total / (1024 ** 3)
used_memory = memory_info.used / (1024 ** 3)
available_memory = memory_info.available / (1024 ** 3)

print(f'Mémoire Totale: {total_memory:.2f} GB')
print(f'Mémoire Utilisée: {used_memory:.2f} GB')
print(f'Mémoire Disponible: {available_memory:.2f} GB')

In [ ]:
# Afficher les tailles des objets en mémoire en parcourant une copie des clés du dictionnaire
all_objects = sys._getframe().f_globals.copy()
for obj_name, obj in all_objects.items():
    print(f'{obj_name}: {sys.getsizeof(obj) / 1024**2:.2f} MB')

In [ ]:
# Libérer du mémoire 
import gc

# Supprimer l'objet volumineux
del imdb1

# Forcer le garbage collector à libérer la mémoire
gc.collect()

In [3]:
# Paths
MovieLens_Imdb_path = 'C:/Users/anacv/OneDrive/Documentos/Data Scientist/00. Projet/Data/DataFrames travaillés/1. Unions/MovieLens_Imdb/MovieLens_Imdb1.feather'
name_basics_path = 'C:/Users/anacv/OneDrive/Documentos/Data Scientist/00. Projet/Data/DataFrames travaillés/0. Exploration/name_basics.feather'
Imdb_path = 'C:/Users/anacv/OneDrive/Documentos/Data Scientist/00. Projet/Data/DataFrames travaillés/1. Unions/Imdb/01_title_basics_crew_ratings.feather'

In [5]:
# Lecture de movieLens_Imdb_path
MovieLens_Imdb = pd.read_feather(MovieLens_Imdb_path )
MovieLens_Imdb.head()

,userId,movieId,rating,date_rating,time_rating,year_rating,month_rating,genres,title_name,release_year,...,imdb_isAdult,imdb_startYear,imdb_endYear,imdb_runtimeMinutes,imdb_genres,imdb_tconst_numeric,imdb_directors,imdb_writers,imdb_averageRating,imdb_numVotes
0,1,2,3.5,2005-04-02,23:53:47,2005,4,Adventure|Children|Fantasy,Jumanji,1995,...,0,1995,<NA>,104.0,"Adventure,Comedy,Family",113497.0,nm0002653,"nm0378144,nm0852430,nm0833164,nm0885575",7.1,384991
1,1,29,3.5,2005-04-02,23:31:16,2005,4,Adventure|Drama|Fantasy|Mystery|Sci-Fi,"City of Lost Children, The (Cité des enfants p...",1995,...,0,1995,<NA>,112.0,"Adventure,Drama,Fantasy",112682.0,"nm0001988,nm0000466","nm0012496,nm0000466,nm0001988,nm0491011",7.5,72399
2,1,32,3.5,2005-04-02,23:33:39,2005,4,Mystery|Sci-Fi|Thriller,Twelve Monkeys (a.k.a. 12 Monkeys),1995,...,0,1995,<NA>,129.0,"Mystery,Sci-Fi,Thriller",114746.0,nm0000416,"nm0003408,nm0672459,nm0672466",8.0,654005
3,1,47,3.5,2005-04-02,23:32:07,2005,4,Mystery|Thriller,Seven (a.k.a. Se7en),1995,...,0,1995,<NA>,127.0,"Crime,Drama,Mystery",114369.0,nm0000399,nm0001825,8.6,1839918
4,1,50,3.5,2005-04-02,23:29:40,2005,4,Crime|Mystery|Thriller,"Usual Suspects, The",1995,...,0,1995,<NA>,106.0,"Crime,Drama,Mystery",114814.0,nm0001741,nm0003160,8.5,1161903


In [11]:
MovieLens_Imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 26 columns):
 #   Column               Dtype         
---  ------               -----         
 0   userId               int64         
 1   movieId              int64         
 2   rating               float64       
 3   date_rating          datetime64[ns]
 4   time_rating          string        
 5   year_rating          int32         
 6   month_rating         int32         
 7   genres               string        
 8   title_name           string        
 9   release_year         Int64         
 10  imdbId               int64         
 11  tmdbId               float64       
 12  imdb_tconst          string        
 13  imdb_titleType       string        
 14  imdb_primaryTitle    string        
 15  imdb_originalTitle   string        
 16  imdb_isAdult         Int64         
 17  imdb_startYear       Int64         
 18  imdb_endYear         Int64         
 19  imdb_runtimeMinutes

In [13]:
MovieLens_Imdb.duplicated('movieId').sum()

19973519

In [7]:
# Sélection des colonnes nécessaires seulement (base de données avec des doublons pour les userids et leurs ratings)
MovieLens_Imdb = MovieLens_Imdb[['imdbId', 'imdb_tconst', 'imdb_directors']]
MovieLens_Imdb.head()

,imdbId,imdb_tconst,imdb_directors
0,113497,tt0113497,nm0002653
1,112682,tt0112682,"nm0001988,nm0000466"
2,114746,tt0114746,nm0000416
3,114369,tt0114369,nm0000399
4,114814,tt0114814,nm0001741


In [9]:
MovieLens_Imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   imdbId          int64 
 1   imdb_tconst     string
 2   imdb_directors  string
dtypes: int64(1), string(2)
memory usage: 457.8 MB


In [11]:
# Afin de travailler avec seulement les lignes nécessaires 
# Supprime les lignes en double dans toutes les colonnes
MovieLens_Imdb = MovieLens_Imdb.drop_duplicates()
MovieLens_Imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26744 entries, 0 to 19989075
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   imdbId          26744 non-null  int64 
 1   imdb_tconst     26694 non-null  string
 2   imdb_directors  26664 non-null  string
dtypes: int64(1), string(2)
memory usage: 835.8 KB


In [13]:
### 1. Extend la colonne 'directors' 
MovieLens_Imdb = MovieLens_Imdb.assign(imdb_directors=MovieLens_Imdb['imdb_directors'].str.split(',')).explode('imdb_directors')
MovieLens_Imdb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30307 entries, 0 to 19989075
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   imdbId          30307 non-null  int64 
 1   imdb_tconst     30257 non-null  string
 2   imdb_directors  30227 non-null  object
dtypes: int64(1), object(1), string(1)
memory usage: 947.1+ KB


In [15]:
### Nettoyage de la colonne 'directors' en prenant en compte les valeurs manquantes
# Supprimer les espaces autour des valeurs non manquantes de 'directors'
MovieLens_Imdb['imdb_directors'] = MovieLens_Imdb['imdb_directors'].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Supprimer tous les caractères invisibles et espaces pour les valeurs non manquantes
MovieLens_Imdb['imdb_directors'] = MovieLens_Imdb['imdb_directors'].apply(lambda x: re.sub(r'\s+', '', x) if isinstance(x, str) else x)

# Convertir explicitement en type string en s'assurant qu'il n'y a pas de NaN dans 'director_code'
MovieLens_Imdb['imdb_directors'] = MovieLens_Imdb['imdb_directors'].astype(str, errors='ignore')

In [17]:
MovieLens_Imdb.head()

,imdbId,imdb_tconst,imdb_directors
0,113497,tt0113497,nm0002653
1,112682,tt0112682,nm0001988
1,112682,tt0112682,nm0000466
2,114746,tt0114746,nm0000416
3,114369,tt0114369,nm0000399


In [19]:
### 2. Lire et fusionner directement imdb1 et name_basics
MovieLens_Imdb = MovieLens_Imdb.merge(
    pd.read_feather(name_basics_path, columns=['nconst', 'primaryName', 'knownForTitles']),
    how='left',
    left_on='imdb_directors',
    right_on='nconst'
)

In [35]:
MovieLens_Imdb.head()

,imdbId,imdb_tconst,imdb_directors,nconst,primaryName,knownForTitles
0,113497,tt0113497,nm0002653,nm0002653,Joe Johnston,"tt0458339,tt0102803,tt0082971,tt0132477"
1,112682,tt0112682,nm0001988,nm0001988,Marc Caro,"tt0101700,tt0112682,tt0118583,tt0487928"
2,112682,tt0112682,nm0000466,nm0000466,Jean-Pierre Jeunet,"tt0344510,tt0211915,tt0101700,tt0112682"
3,114746,tt0114746,nm0000416,nm0000416,Terry Gilliam,"tt0088846,tt0071853,tt1054606,tt0081633"
4,114369,tt0114369,nm0000399,nm0000399,David Fincher,"tt0114369,tt0443706,tt2267998,tt1285016"


In [21]:
# Supprimer la colonne 'nconst' du résultat final
MovieLens_Imdb = MovieLens_Imdb.drop(columns=['nconst'])

# Renommer les colonnes 'primaryName' et 'knownForTitles' dans le DataFrame résultant
MovieLens_Imdb = MovieLens_Imdb.rename(columns={
    'primaryName': 'directors_primaryName',
    'knownForTitles': 'directors_knownForTitles'
})

MovieLens_Imdb.head()

,imdbId,imdb_tconst,imdb_directors,directors_primaryName,directors_knownForTitles
0,113497,tt0113497,nm0002653,Joe Johnston,"tt0458339,tt0102803,tt0082971,tt0132477"
1,112682,tt0112682,nm0001988,Marc Caro,"tt0101700,tt0112682,tt0118583,tt0487928"
2,112682,tt0112682,nm0000466,Jean-Pierre Jeunet,"tt0344510,tt0211915,tt0101700,tt0112682"
3,114746,tt0114746,nm0000416,Terry Gilliam,"tt0088846,tt0071853,tt1054606,tt0081633"
4,114369,tt0114369,nm0000399,David Fincher,"tt0114369,tt0443706,tt2267998,tt1285016"


In [23]:
### 3. Regrouper les noms des directeurs par film

# Fonction d'agrégation pour pandas
def aggregate_partition(pdf):
    return pdf.groupby('imdbId').agg({
        'imdb_tconst': 'first',
        'imdb_directors': lambda x: ','.join(x.dropna()),  # Joindre les valeurs non-nulles de 'directors'
        'directors_primaryName': lambda x: ','.join(x.dropna()),  # Joindre les valeurs non-nulles de 'directors_primaryName'
        'directors_knownForTitles': lambda x: ','.join(x.dropna()),  # Joindre les valeurs non-nulles de 'directors_knownForTitles'
    })


In [25]:
# Application de la fonction d'agrégation 
MovieLens_Imdb_aggregated = aggregate_partition(MovieLens_Imdb)

In [27]:
MovieLens_Imdb_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26744 entries, 5 to 4530184
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   imdb_tconst               26694 non-null  string
 1   imdb_directors            26744 non-null  object
 2   directors_primaryName     26744 non-null  object
 3   directors_knownForTitles  26744 non-null  object
dtypes: object(3), string(1)
memory usage: 1.0+ MB


In [29]:
# Afin d'identifier les vides des colonnes agregées
# Remplacement des valeurs dans plusieurs colonnes
MovieLens_Imdb_aggregated.replace({
    'imdb_directors': {'<NA>': pd.NA},
    'directors_primaryName': {'': pd.NA},
    'directors_knownForTitles': {'': pd.NA}
}, inplace=True)

MovieLens_Imdb_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26744 entries, 5 to 4530184
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   imdb_tconst               26694 non-null  string
 1   imdb_directors            26664 non-null  object
 2   directors_primaryName     26664 non-null  object
 3   directors_knownForTitles  26664 non-null  object
dtypes: object(3), string(1)
memory usage: 1.0+ MB


In [ ]:
# Vérif des valeurs manquantes
MovieLens_Imdb_aggregated[MovieLens_Imdb_aggregated['imdb_tconst'].isnull()]

In [31]:
# Appliquer reset_index() pour remettre `imdbId` comme colonne normale au lieu d'index
MovieLens_Imdb_aggregated = MovieLens_Imdb_aggregated.reset_index()

In [33]:
MovieLens_Imdb_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26744 entries, 0 to 26743
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   imdbId                    26744 non-null  int64 
 1   imdb_tconst               26694 non-null  string
 2   imdb_directors            26664 non-null  object
 3   directors_primaryName     26664 non-null  object
 4   directors_knownForTitles  26664 non-null  object
dtypes: int64(1), object(3), string(1)
memory usage: 1.0+ MB


In [75]:
MovieLens_Imdb_aggregated.head()

,imdbId,imdb_tconst,imdb_directors,directors_primaryName,directors_knownForTitles
0,5,tt0000005,nm0005690,William K.L. Dickson,"tt1428455,tt1496763,tt0308254,tt0219560"
1,8,tt0000008,nm0005690,William K.L. Dickson,"tt1428455,tt1496763,tt0308254,tt0219560"
2,10,tt0000010,nm0525910,Louis Lumière,"tt6231346,tt1167665,tt0000014,tt2461054"
3,12,tt0000012,"nm0525908, nm0525910","Auguste Lumière, Louis Lumière","tt8485720,tt1982760,tt32220357,tt0000012, tt62..."
4,14,tt0000014,nm0525910,Louis Lumière,"tt6231346,tt1167665,tt0000014,tt2461054"


In [ ]:
######### Part II. Recherche des noms de knownForTitles

In [35]:
### 1. Extend la colonne 'directors_knownForTitles' 
MovieLens_Imdb_aggtd2 = MovieLens_Imdb_aggregated.assign(directors_knownForTitles=MovieLens_Imdb_aggregated['directors_knownForTitles'].str.split(',')).explode('directors_knownForTitles')
MovieLens_Imdb_aggtd2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119688 entries, 0 to 26743
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   imdbId                    119688 non-null  int64 
 1   imdb_tconst               119638 non-null  string
 2   imdb_directors            119608 non-null  object
 3   directors_primaryName     119608 non-null  object
 4   directors_knownForTitles  119608 non-null  object
dtypes: int64(1), object(3), string(1)
memory usage: 5.5+ MB


In [37]:
### Nettoyage de la colonne 'directors_knownForTitles' en prenant en compte les valeurs manquantes
# Supprimer les espaces autour des valeurs non manquantes de 'directors'
MovieLens_Imdb_aggtd2['directors_knownForTitles'] = MovieLens_Imdb_aggtd2['directors_knownForTitles'].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Supprimer tous les caractères invisibles et espaces pour les valeurs non manquantes
MovieLens_Imdb_aggtd2['directors_knownForTitles'] = MovieLens_Imdb_aggtd2['directors_knownForTitles'].apply(lambda x: re.sub(r'\s+', '', x) if isinstance(x, str) else x)

# Convertir explicitement en type string en s'assurant qu'il n'y a pas de NaN dans 'director_code'
MovieLens_Imdb_aggtd2['directors_knownForTitles'] = MovieLens_Imdb_aggtd2['directors_knownForTitles'].astype(str, errors='ignore')


In [39]:
# Vérification des doublons 
doublons = MovieLens_Imdb_aggtd2.duplicated().any()
print("Y a-t-il des doublons dans le DataFrame ? ", doublons)

Y a-t-il des doublons dans le DataFrame ?  True


In [125]:
MovieLens_Imdb_aggtd2[MovieLens_Imdb_aggtd2.duplicated(keep=False)]

,imdbId,imdb_tconst,imdb_directors,directors_primaryName,directors_knownForTitles
15,2461,tt0002461,"nm0130633, nm0443857","André Calmettes, James Keane",tt0002461
15,2461,tt0002461,"nm0130633, nm0443857","André Calmettes, James Keane",tt0002461
19,3419,tt0003419,"nm0263912, nm0753233","Hanns Heinz Ewers, Stellan Rye",tt0003419
19,3419,tt0003419,"nm0263912, nm0753233","Hanns Heinz Ewers, Stellan Rye",tt0003419
20,3489,tt0003489,"nm0143333, nm0734958","Mario Caserini, Eleuterio Rodolfi",tt0003489
...,...,...,...,...,...
26708,4006794,tt4006794,"nm0105001, nm0148999","Spike Brandt, Tony Cervone",tt1660055
26709,4007502,tt4007502,"nm0118333, nm1601644","Chris Buck, Jennifer Lee",tt2294629
26709,4007502,tt4007502,"nm0118333, nm1601644","Chris Buck, Jennifer Lee",tt4520988
26709,4007502,tt4007502,"nm0118333, nm1601644","Chris Buck, Jennifer Lee",tt2294629


In [127]:
MovieLens_Imdb_aggregated[MovieLens_Imdb_aggregated['imdbId']==2461]

,imdbId,imdb_tconst,imdb_directors,directors_primaryName,directors_knownForTitles
15,2461,tt0002461,"nm0130633, nm0443857","André Calmettes, James Keane","tt0001175,tt3096726,tt0237838,tt0002461, tt000..."


In [129]:
# Visualisation des données

# Ajuster l'option pour afficher tout le contenu des cellules
pd.set_option('display.max_colwidth', None)

# Filtrer pour trouver la ligne avec imdbId égal à 2461 et afficher uniquement la colonne `directors_knownForTitles`
resultat = MovieLens_Imdb_aggregated.loc[MovieLens_Imdb_aggregated['imdbId'] == 2461, 'directors_knownForTitles']

# Afficher le contenu de la cellule
print(resultat)

# Rétablir la configuration par défaut
pd.reset_option('display.max_colwidth')

# Commentaire : 
# Il peut avoir des directors connus par le même film
# Il faut éliminer les doublons

15    tt0001175,tt3096726,tt0237838,tt0002461, tt0002461,tt0005763,tt0179517,tt0190031
Name: directors_knownForTitles, dtype: object


In [41]:
# Supprime les lignes en double dans toutes les colonnes
MovieLens_Imdb_aggtd2 = MovieLens_Imdb_aggtd2.drop_duplicates()
MovieLens_Imdb_aggtd2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116273 entries, 0 to 26743
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   imdbId                    116273 non-null  int64 
 1   imdb_tconst               116223 non-null  string
 2   imdb_directors            116193 non-null  object
 3   directors_primaryName     116193 non-null  object
 4   directors_knownForTitles  116273 non-null  object
dtypes: int64(1), object(3), string(1)
memory usage: 5.3+ MB


In [133]:
MovieLens_Imdb_aggtd2.head()

,imdbId,imdb_tconst,imdb_directors,directors_primaryName,directors_knownForTitles
0,5,tt0000005,nm0005690,William K.L. Dickson,tt1428455
0,5,tt0000005,nm0005690,William K.L. Dickson,tt1496763
0,5,tt0000005,nm0005690,William K.L. Dickson,tt0308254
0,5,tt0000005,nm0005690,William K.L. Dickson,tt0219560
1,8,tt0000008,nm0005690,William K.L. Dickson,tt1428455


In [43]:
### 2. Lire et fusionner directement imdb1 et name_basics

MovieLens_Imdb_aggtd2 = MovieLens_Imdb_aggtd2.merge(
    pd.read_feather(Imdb_path, columns=['tconst', 'primaryTitle']),
    how='left',
    left_on='directors_knownForTitles',
    right_on='tconst'
)

MovieLens_Imdb_aggtd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116273 entries, 0 to 116272
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   imdbId                    116273 non-null  int64 
 1   imdb_tconst               116223 non-null  string
 2   imdb_directors            116193 non-null  object
 3   directors_primaryName     116193 non-null  object
 4   directors_knownForTitles  116273 non-null  object
 5   tconst                    116193 non-null  string
 6   primaryTitle              116193 non-null  string
dtypes: int64(1), object(3), string(3)
memory usage: 6.2+ MB


In [45]:
# Supprimer la colonne 'tconst' du résultat final
MovieLens_Imdb_aggtd2 = MovieLens_Imdb_aggtd2.drop(columns=['tconst'])

# Renommer les colonnes 'primaryTitle' de  'directors_knownForTitles'
MovieLens_Imdb_aggtd2 = MovieLens_Imdb_aggtd2.rename(columns={
    'primaryTitle': 'directors_knownForTitles_primaryTitle'
})

MovieLens_Imdb_aggtd2.head()

,imdbId,imdb_tconst,imdb_directors,directors_primaryName,directors_knownForTitles,directors_knownForTitles_primaryTitle
0,5,tt0000005,nm0005690,William K.L. Dickson,tt1428455,Sandow
1,5,tt0000005,nm0005690,William K.L. Dickson,tt1496763,Dr. Colton or Dentist Scene
2,5,tt0000005,nm0005690,William K.L. Dickson,tt0308254,Fencing Contest from 'The Three Musketeers'
3,5,tt0000005,nm0005690,William K.L. Dickson,tt0219560,The Boxing Cats (Prof. Welton's)
4,8,tt0000008,nm0005690,William K.L. Dickson,tt1428455,Sandow


In [47]:
# Vérification des doublons 
doublons = MovieLens_Imdb_aggtd2.duplicated().any()
print("Y a-t-il des doublons dans le DataFrame ? ", doublons)

Y a-t-il des doublons dans le DataFrame ?  False


In [49]:
### 3. Regrouper par film

# Fonction d'agrégation pour pandas
def aggregate_partition(pdf):
    return pdf.groupby('imdbId').agg({
        'imdb_tconst': 'first',
        'imdb_directors': 'first',
        'directors_primaryName': 'first',
        'directors_knownForTitles': : lambda x: ','.join(x.dropna()),
        'directors_knownForTitles_primaryTitle': lambda x: ','.join(x.dropna())  # Joindre les valeurs non-nulles de 'directors_knownForTitles'
    })


In [51]:
# Application de la fonction d'agrégation 
MovieLens_Imdb_aggtd2 = aggregate_partition(MovieLens_Imdb_aggtd2)

In [53]:
MovieLens_Imdb_aggtd2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26744 entries, 5 to 4530184
Data columns (total 5 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   imdb_tconst                            26694 non-null  string
 1   imdb_directors                         26664 non-null  object
 2   directors_primaryName                  26664 non-null  object
 3   directors_knownForTitles               26744 non-null  object
 4   directors_knownForTitles_primaryTitle  26744 non-null  string
dtypes: object(3), string(2)
memory usage: 1.2+ MB


In [ ]:
# Afin d'identifier les vides des colonnes agregées
# Remplacement des valeurs dans plusieurs colonnes
MovieLens_Imdb_aggregated.replace({
    'imdb_directors': {'<NA>': pd.NA},
    'directors_primaryName': {'': pd.NA},
    'directors_knownForTitles': {'': pd.NA}
}, inplace=True)

MovieLens_Imdb_aggregated.info()